In [1]:
import gurobipy as gp
print(gp.gurobi.version())

(12, 0, 2)


In [2]:
import numpy as np
import pulp
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional
import itertools
from scipy.spatial.transform import Rotation

@dataclass
class AtomType:
    symbol: str
    charge: float
    A: float
    rho: float
    C: float
    shannon_radius: float

@dataclass
class GridSite:
    position: np.ndarray
    fractional_coords: np.ndarray
    site_id: int

class CubicGrid:
    def __init__(self, lattice_param: float, grid_divisions: int):
        self.a = lattice_param
        self.divisions = grid_divisions
        self.spacing = lattice_param / grid_divisions
        self.sites = self._generate_sites()
        
    def _generate_sites(self) -> List[GridSite]:
        sites = []
        idx = 0
        for i, j, k in itertools.product(range(self.divisions), repeat=3):
            pos = np.array([i, j, k]) * self.spacing
            frac = np.array([i, j, k]) / self.divisions
            sites.append(GridSite(pos, frac, idx))
            idx += 1
        return sites
    
    def periodic_distance(self, pos1: np.ndarray, pos2: np.ndarray) -> float:
        delta = pos1 - pos2
        for dim in range(3):
            if delta[dim] > self.a / 2:
                delta[dim] -= self.a
            elif delta[dim] < -self.a / 2:
                delta[dim] += self.a
        return np.linalg.norm(delta)

def buckingham_energy(r: float, A: float, rho: float, C: float) -> float:
    if r < 1e-6:
        return 0
    return A * np.exp(-r / rho) - C / r**6

KE_EV_ANG = 14.3996
eps_r = 10.0

def ewald_energy(q1: float, q2: float, r: float) -> float:
    if r < 1e-6:
        return 0
    return KE_EV_ANG * q1 * q2 / (eps_r * r)

def compute_interaction_matrix(
        grid: CubicGrid,
        atom_types: Dict[str, AtomType],
        pair_params: Dict[Tuple[str, str], Tuple[float, float, float]],
        r_cutoff: float = 6.0
) -> Dict[Tuple[int, str, int, str], float]:
    interaction = {}
    n_sites = len(grid.sites)
    positions = [site.position for site in grid.sites]
    
    for i, j in itertools.combinations(range(n_sites), 2):
        r = grid.periodic_distance(positions[i], positions[j])
        if r > r_cutoff:
            continue

        for e1, e2 in itertools.product(atom_types.keys(), repeat=2):
            if (e1, e2) in pair_params:
                A, rho, C = pair_params[(e1, e2)]
            elif (e2, e1) in pair_params:
                A, rho, C = pair_params[(e2, e1)]
            else:
                at1, at2 = atom_types[e1], atom_types[e2]
                A   = (at1.A + at2.A) / 2
                rho = (at1.rho + at2.rho) / 2
                C   = (at1.C + at2.C) / 2

            e_buck  = buckingham_energy(r, A, rho, C)
            q1, q2  = atom_types[e1].charge, atom_types[e2].charge
            e_ewald = ewald_energy(q1, q2, r)
            interaction[(i, e1, j, e2)] = e_buck + e_ewald
                
    return interaction

def get_symmetry_orbits(grid: CubicGrid, space_group: str = "Pm-3m") -> List[List[int]]:
    # Generate 48 symmetry operations for cubic Pm-3m group
    symmetry_ops = []
    perms = list(itertools.permutations([0, 1, 2]))
    sign_prods = list(itertools.product([-1, 1], repeat=3))
    
    for perm in perms:
        for signs in sign_prods:
            mat = np.zeros((3, 3))
            for i in range(3):
                mat[i, perm[i]] = signs[i]
            symmetry_ops.append(mat)
    
    orbits = []
    visited = set()
    grid_size = grid.divisions
    
    for site in grid.sites:
        if site.site_id in visited:
            continue
        orbit = [site.site_id]
        visited.add(site.site_id)
        
        for op in symmetry_ops:
            new_frac = op @ site.fractional_coords
            new_frac = new_frac % 1.0
            i_new = int(round(new_frac[0] * grid_size)) % grid_size
            j_new = int(round(new_frac[1] * grid_size)) % grid_size
            k_new = int(round(new_frac[2] * grid_size)) % grid_size
            new_id = i_new * (grid_size**2) + j_new * grid_size + k_new
            
            if new_id not in visited:
                orbit.append(new_id)
                visited.add(new_id)
        
        orbits.append(orbit)
    
    return orbits

def add_symmetry_constraints(
        prob: pulp.LpProblem,
        x: Dict[Tuple[int, str], pulp.LpVariable],
        orbits: List[List[int]],
        atom_types: Dict[str, AtomType]
):
    for orbit in orbits:
        if len(orbit) < 2:
            continue
        for e in atom_types:
            ref_var = x[(orbit[0], e)]
            for site_idx in orbit[1:]:
                prob += x[(site_idx, e)] == ref_var

def add_proximity_constraints(
        prob: pulp.LpProblem,
        grid: CubicGrid,
        x: Dict[Tuple[int, str], pulp.LpVariable],
        atom_types: Dict[str, AtomType],
        min_distance_factor: float = 0.7
):
    positions = [site.position for site in grid.sites]
    n_sites = len(positions)
    
    for i, j in itertools.combinations(range(n_sites), 2):
        r = grid.periodic_distance(positions[i], positions[j])
        for e1 in atom_types:
            r1 = atom_types[e1].shannon_radius
            for e2 in atom_types:
                r2 = atom_types[e2].shannon_radius
                min_dist = min_distance_factor * (r1 + r2)
                if r < min_dist:
                    prob += x[(i, e1)] + x[(j, e2)] <= 1

def solve_global_ip(
        grid: CubicGrid, 
        atom_types: Dict[str, AtomType], 
        composition: Dict[str, int],
        space_group: Optional[str] = None
) -> List[Tuple[str, np.ndarray]]:
    prob = pulp.LpProblem("Global_CSP", pulp.LpMinimize)
    n_sites = len(grid.sites)
    
    # Decision variables
    x = pulp.LpVariable.dicts("x", 
        ((i, e) for i in range(n_sites) for e in atom_types),
        cat='Binary'
    )
    
    # Constraints
    for i in range(n_sites):
        prob += pulp.lpSum(x[(i, e)] for e in atom_types) <= 1

    for e, count in composition.items():
        prob += pulp.lpSum(x[(i, e)] for i in range(n_sites)) == count

    # Interaction matrix with reduced cutoff
    interaction = compute_interaction_matrix(grid, atom_types, pair_params, r_cutoff=3.0)
    
    # Proximity constraints
    add_proximity_constraints(prob, grid, x, atom_types, min_distance_factor=0.75)
    
    # Symmetry constraints
    if space_group:
        orbits = get_symmetry_orbits(grid, space_group)
        add_symmetry_constraints(prob, x, orbits, atom_types)
    
    # Linearized objective function
    z = pulp.LpVariable.dicts("z", 
        ((i, e1, j, e2) for (i, e1, j, e2) in interaction if i < j),
        cat='Binary'
    )
    
    for (i, e1, j, e2) in z:
        prob += z[(i, e1, j, e2)] <= x[(i, e1)]
        prob += z[(i, e1, j, e2)] <= x[(j, e2)]
        prob += z[(i, e1, j, e2)] >= x[(i, e1)] + x[(j, e2)] - 1

    prob += pulp.lpSum(
        interaction[key] * z[key] 
        for key in z if key in interaction
    )

    # Solver configuration
    solver = pulp.GUROBI(
        msg=True,
        timeLimit=300,
        mipFocus=1,
        presolve=2,
        heuristics=0.5
    )
    
    # Solve
    prob.solve(solver)
    print('Status:', pulp.LpStatus[prob.status])
    
    # Extract results
    selected = []
    for (i, e) in x:
        if pulp.value(x[(i, e)]) > 0.5:
            selected.append((e, grid.sites[i].position))
    
    return selected

# === Parameter Setup ===
lattice_param = 4.0
grid_divisions = 4  # Reduced for feasibility
space_group = "Pm-3m"
composition = {'Sr': 1, 'Ti': 1, 'O': 3}

atom_types = {
    'Sr': AtomType('Sr', 2.0, 1450.0, 0.35, 0.0, 1.18),
    'Ti': AtomType('Ti', 4.0, 1000.0, 0.35, 0.0, 0.42),
    'O':  AtomType('O', -2.0, 22764.0, 0.149, 27.88, 1.35)
}

pair_params = {
    ('Sr','O'): (1455.2, 0.3585, 0.0),
    ('Ti','O'): (3147.6, 0.2870, 0.0),
    ('O','O') : (22764.0, 0.1490, 27.88),
    ('Sr','Sr'): (2386.0, 0.3327, 0.0),
    ('Ti','Ti'): (2000.0, 0.3500, 0.0),
    ('Sr','Ti'): (1155.0, 0.3560, 0.0)
}

# === Main Execution ===
if __name__ == "__main__":
    print("Creating crystal grid...")
    grid = CubicGrid(lattice_param, grid_divisions)
    
    print("Solving for globally optimal structure...")
    selected_atoms = solve_global_ip(
        grid, 
        atom_types, 
        composition,
        space_group=space_group
    )
    
    print("\nPredicted atomic positions:")
    for atom in selected_atoms:
        element, pos = atom
        print(f"{element}: {np.round(pos, 4)}")

Set parameter Username
Set parameter LicenseID to value 2677474
Academic license - for non-commercial use only - expires 2026-06-12
Set parameter Username
Set parameter LicenseID to value 2677474
Academic license - for non-commercial use only - expires 2026-06-12
Set parameter Username
Set parameter LicenseID to value 2677474
Academic license - for non-commercial use only - expires 2026-06-12
Set parameter Username
Set parameter LicenseID to value 2677474
Academic license - for non-commercial use only - expires 2026-06-12
Set parameter Username
Set parameter LicenseID to value 2677474
Academic license - for non-commercial use only - expires 2026-06-12
Set parameter Username
Set parameter LicenseID to value 2677474
Academic license - for non-commercial use only - expires 2026-06-12
Creating crystal grid...
Solving for globally optimal structure...
Set parameter Username
Set parameter LicenseID to value 2677474
Academic license - for non-commercial use only - expires 2026-06-12
Set param

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# === Expecting that `selected_atoms` has been produced by the solver ===
# selected_atoms is a list of tuples: (element_symbol, np.ndarray([x, y, z]))
# For demonstration, I'll create a dummy list if it doesn't exist.

# Split positions and element labels
chosen_elements = [elem for elem, pos in selected_atoms]
chosen_positions = [pos for elem, pos in selected_atoms]

# === Visualization Parameters ===
lattice_parameter = 5.0   # Update if your solver used a different value
grid_divisions = 4
spacing = lattice_parameter / grid_divisions

# 1. Draw 3D scatter
fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111, projection='3d')

unique_types = sorted(set(chosen_elements))
markers = {'Sr': 'o', 'Ti': '^', 'O': 's'}
for elem in unique_types:
    idxs = [i for i, t in enumerate(chosen_elements) if t == elem]
    pts = np.array([chosen_positions[i] for i in idxs])
    ax.scatter(pts[:, 0], pts[:, 1], pts[:, 2],
               label=elem,
               s=120,
               marker=markers.get(elem, 'o'),
               edgecolors='black')

# Draw cube edges
corners_frac = np.array([[i, j, k] for i in [0, 1] for j in [0, 1] for k in [0, 1]])
corners = corners_frac * lattice_parameter
for a in range(8):
    for b in range(a + 1, 8):
        if np.sum(np.abs(corners_frac[a] - corners_frac[b])) == 1:
            p1, p2 = corners[a], corners[b]
            ax.plot(*zip(p1, p2), color='gray')

ax.set_xlabel('X (Å)')
ax.set_ylabel('Y (Å)')
ax.set_zlabel('Z (Å)')
ax.set_title('Atom Distribution in Cubic Cell')
ax.legend()
plt.tight_layout()
plt.show()


ModuleNotFoundError: No module named 'matplotlib'